In [3]:
import torch
import torch.nn as nn
import numpy as np
import random

with open("studyText/dataset.txt") as t:
    text = t.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
print('Уникальные символы:', ''.join(chars))
print(f'Размер словаря: {vocab_size}', '\n')

char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}


def encode(s): return [char_to_idx[c] for c in s]
def decode(l): return ''.join([idx_to_char[i] for i in l])


Уникальные символы: 
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~©«¬¯°²´·»½¿Ë×Øéëìóö÷ĘęśźŻżƍǝɐɓɔɯʁʎʖʚʞ̴̵̸̡̢̧̨̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̹̺̻̼͇͈͉͍͎͓͔͕͖͙͚̀́̂̃̄̅̆̇̈̉̋̌̍̎̏̐̑̒̓̔̽̾̀́͂̓̈́͆͊͋͌͐͑͒͗͛ͣͥͧͪͬͮͯ̕̚͘͜͝͠͡ͅΒΗΜΠδεοπЁІЇАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёєії҉აბგდევზთიკლმნორსტუფქღყშჩცწხჯჰᲠ​‍‐–—‘’‚“”•…″⁃₂₃€№⇄−√≈Ⓘⓘ█▒☝☺☻♂⚡⚰✉❗❤⠁⠂⠅⠇⠊⠍⠏⠑⠕⠗⠚⠝⠥⠫⠱⠲⠵。あかくしせたとなにはまもをん为举事人代任会副卐名命已并应成手接数新最果然犯理由的直私经结给翻老虽表訳誰选长️﻿，－🇱🇵🌈🌚🌝🌲🍎🍏🍬🎅🎉🏪🏳🏻🏼🐌🐻👀👇👈👉👞👣👦👨👩💌💨💭💰💺🔞🔥🕺😂😃😅😈😉😊😋😍😎😏😓😜😭😱😳😹😻🙏🚗🤙🤝🤡🤣🤯🤵🤷🥀🥃🥺🫡
Размер словаря: 514 



In [14]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=2, dropout=0.2):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        
        output, hidden = self.rnn(embedded, hidden)

        output = self.fc(output)
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        h = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return (h, c)

hidden_size = 256
model = CharRNN(vocab_size, hidden_size)
print(model)

CharRNN(
  (embedding): Embedding(514, 256)
  (rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=514, bias=True)
)


In [10]:
data = encode(text)
data = torch.tensor(data, dtype=torch.long)


seq_length = 50
batch_size = 64

def get_batch():
    start_idxs = torch.randint(0, len(data) - seq_length, (batch_size,))
    
    x_batch = torch.stack([data[i:i+seq_length] for i in start_idxs])
    y_batch = torch.stack([data[i+1:i+seq_length+1] for i in start_idxs])
    
    return x_batch, y_batch

x, y = get_batch()
print("Входная последовательность:", x[0][:10], "...")
print("Целевая последовательность:", y[0][:10], "...")
print("Текст входа:", decode(x[0][:10].tolist()))
print("Текст цели:", decode(y[0][:10].tolist()))

Входная последовательность: tensor([287, 284, 276, 287,   1, 281,   1, 294, 287, 289]) ...
Целевая последовательность: tensor([284, 276, 287,   1, 281,   1, 294, 287, 289, 287]) ...
Текст входа: олго и хор
Текст цели: лго и хоро


In [ ]:
learning_rate = 0.0001
num_epochs = 20000
print_interval = 1000

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(num_epochs):
    x_batch, y_batch = get_batch()
    
    optimizer.zero_grad()

    hidden = model.init_hidden(batch_size)
    
    output, hidden = model(x_batch, hidden)
    
    loss = criterion(output.view(-1, vocab_size), y_batch.view(-1))
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    
    if epoch % print_interval == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
        

print("Обучение завершено!")

Epoch 0, Loss: 6.2542


In [ ]:
def generate_text(model, start_str, length=100, temperature=0.8):
    model.eval()
    
    chars = [ch for ch in start_str]
    input_seq = torch.tensor(encode(chars), dtype=torch.long).unsqueeze(0)
    hidden = model.init_hidden(1)
    
    for _ in range(length):
        with torch.no_grad():
            output, hidden = model(input_seq, hidden)
            
            logits = output[0, -1, :] / temperature
            probabilities = torch.softmax(logits, dim=0)
            
            next_char_idx = torch.multinomial(probabilities, 1).item()
            
            chars.append(idx_to_char[next_char_idx])
            input_seq = torch.tensor([[next_char_idx]], dtype=torch.long)
    
    return ''.join(chars)

model.eval()
generated_text = generate_text(model, "Трава", 100)
print("Сгенерированный текст:")
print(generated_text)

NameError: name 'model' is not defined

In [ ]:
def chat_with_model(model, initial_prompt="Привет", response_length=50):
    """Функция для 'общения' с моделью"""
    print("Вы:", initial_prompt)
    
    response = generate_text(model, initial_prompt, response_length)

    generated_response = response[len(initial_prompt):]
    print("Модель:", generated_response)

chat_with_model(model, "костер")

NameError: name 'model' is not defined